In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Multi-fidelity (MF) optimization

In most cases it is better to do many cheap evaluations of an approximation to the target function than it is to only optimize the target function. This example demonstrates the 'multi-fidelity' capabilities of xopt. 

We follow the implementation of multi-fidelity bayesian optimization used in botorch https://botorch.org/tutorials/multi_fidelity_bo to optimize the synthetic test function AugmentedHartmann https://botorch.org/api/test_functions.html.

The difference between normal Bayesian optimization and MF optimization is that we specify a 'cost' to making observations at a given fidelity. For this example we assume a base cost of 5 and a fidelity cost between 0-1. The algorithm should make many observations at lower fidelity relative to higher fidelity, lowering the total observation cost. 

NOTE: The cost parameter is required to be the LAST element of the variables list. Also this method is best suited for parallel observations of the test function.

In [2]:
# Import the class
from xopt import Xopt
from xopt.bayesian.generators.multi_fidelity import MultiFidelityGenerator
from xopt.bayesian.models.models import create_multi_fidelity_model
from botorch.test_functions.multi_fidelity import AugmentedHartmann


The `Xopt` object can be instantiated from a JSON or YAML file, or a dict, with the proper structure.

Here we will make one

In [3]:
import yaml
# Make a proper input file. 
YAML = """
xopt: {output_path: null, verbose: true}

algorithm:
  name: multi_fidelity
  options:  
      n_initial_samples: 16
      n_steps: 10
      verbose: True
      use_gpu: False
      generator_options:                     ## options for bayesian exploration acquisition function
          batch_size: 4                      ## batch size for parallelized optimization
          acq: custom_acq.acq                ## custom acquisition function if needed
          fixed_cost: 5.0                    ## fixed cost added onto fidelity cost


simulation: 
  name: test_multi_fidelity
  evaluate: xopt.evaluators.test_multi_fidelity.evaluate

vocs:
  name: test_multi_fidelity
  description: null
  simulation: test_multi_fidelity
  templates: null
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]                          ## NOTE: THIS IS REQUIRED FOR MULTI-FIDELITY OPTIMIZATION
  objectives:
    y1: 'MINIMIZE'
  linked_variables: {}
  constants: {a: dummy_constant}

"""
config = yaml.safe_load(YAML)

In [4]:
X = Xopt(config)
X

Loading config as dict.



            Xopt 
________________________________           
Version: 0.4.3+92.ga35bcf3.dirty
Configured: True
Config as YAML:
xopt: {output_path: null, verbose: true, algorithm: cnsga}
algorithm:
  name: multi_fidelity
  function: xopt.bayesian.algorithms.multi_fidelity_optimize
  options:
    n_initial_samples: 16
    n_steps: 10
    verbose: true
    use_gpu: false
    generator_options: {batch_size: 4, acq: !!python/name:custom_acq.acq '', fixed_cost: 5.0}
    custom_model: !!python/name:xopt.bayesian.models.models.create_multi_fidelity_model ''
    restart_file: null
    initial_x: null
simulation:
  name: test_multi_fidelity
  evaluate: xopt.evaluators.test_multi_fidelity.evaluate
  options: {extra_option: abc}
vocs:
  name: test_multi_fidelity
  description: null
  simulation: test_multi_fidelity
  templates: null
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]
  objectives: {y1: MINIMIZE}
  

# Run BayesOpt

In [5]:
# Pick one of these
from concurrent.futures import ThreadPoolExecutor as PoolExecutor
#from concurrent.futures import ProcessPoolExecutor as PoolExecutor

executor = PoolExecutor()
# This will also work. 
#executor=None

In [6]:
# Change max generations
X.run(executor=executor)
results = X.results

Starting at time 2021-08-23T11:40:08-05:00
<class 'function'>
started running optimization with generator: <xopt.bayesian.generators.multi_fidelity.MultiFidelityGenerator object at 0x00000193FEA67F10>
submitting initial candidates at time 2021-08-23T11:40:08-05:00
starting optimization loop
Model creation time: 0.132 s
Candidate generation time: 24.5 s
Candidate(s): tensor([[0.1445, 0.7707, 0.2275, 0.3497, 0.4089, 0.1356, 0.5591],
        [0.2800, 0.8747, 0.1747, 0.4736, 0.4827, 0.2292, 0.5830],
        [0.1684, 0.8617, 0.0270, 0.2846, 0.4888, 0.2897, 0.5406],
        [0.2594, 0.7251, 0.0403, 0.4023, 0.3323, 0.2560, 0.5287]],
       dtype=torch.float64)
submitting candidates at time 2021-08-23T11:40:32-05:00
Model creation time: 0.144 s
Candidate generation time: 27.34 s
Candidate(s): tensor([[0.3293, 0.9204, 0.1236, 0.5049, 0.4745, 0.2016, 0.5343],
        [0.2990, 0.7272, 0.2154, 0.5186, 0.5282, 0.1847, 0.5658],
        [0.9994, 0.7731, 0.7041, 0.7553, 0.5915, 0.7916, 0.6409],
      

### Get highest fidelity global optimum

In [7]:
# create generator object
gen = MultiFidelityGenerator(X.vocs)

In [8]:
# create model
model = create_multi_fidelity_model(results['variables'], results['corrected_objectives'], results['corrected_constraints'], X.vocs)

In [9]:
## NOTE: we want to get the minimum evaluated at the highest fidelity -> make sure to use get_recommendation
rec = gen.get_recommendation(model)
problem = AugmentedHartmann(negate=False)
problem(rec) ## NOTE: the correct global minimum is -3.32237

tensor([-3.1795])